In [1]:
use marketcamp;

Query OK

## Query para visualizar o fluxo de caixa diario
As variáveis abaixo representam valores setados no código cliente

In [2]:
SET @lojaAtiva = 1;
SET @dateFrom = '2021-01-01';
SET @dateUntil = '2021-07-31';

WITH entradas as (
SELECT DATE(base.dataMovimentacao) as data, sum(base.valorTotal) as valor_entrada
    from marketcamp.movimentacaofinanceiraentrada as derived
    inner join marketcamp.movimentacaofinanceira as base
    on derived.movimentacaoID=base.Id
    where base.lojaID=@lojaAtiva and DATE(base.dataMovimentacao) between @dateFrom and @dateUntil
    group by data
),

saidas as (
SELECT DATE(base.dataMovimentacao) as data, sum(base.valorParcela) as valor_saida
    from marketcamp.movimentacaofinanceirasaida as derived
    inner join marketcamp.movimentacaofinanceira as base
    on derived.movimentacaoID=base.Id
    where base.lojaID=@lojaAtiva and DATE(base.dataMovimentacao) between @dateFrom and @dateUntil
    group by data
),
resumo as
(
    (select entradas.data as data, entradas.valor_entrada as valor_entrada, 0 as valor_saida from entradas)
    UNION ALL
    (select saidas.data, 0, saidas.valor_saida from saidas)
)

select data, sum(valor_entrada), sum(valor_saida), sum(valor_entrada)- sum(valor_saida) as liquido
from resumo
group by data
order by data;


data,sum(valor_entrada),sum(valor_saida),liquido
2021-01-01,2550.00,0.00,2550.00
2021-01-18,660.00,0.00,660.00
2021-03-04,200.00,0.00,200.00
2021-04-03,0.00,300.00,-300.00
2021-04-21,0.00,550.00,-550.00
2021-07-30,0.00,200.00,-200.00


## Query para visualizar o fluxo de caixa mensal
As variáveis abaixo representam valores setados no código cliente

In [3]:
SET @lojaAtiva = 1;
SET @dateFrom = '2021-01-01';
SET @dateUntil = '2021-07-31';

WITH entradas as (
SELECT EXTRACT(YEAR_MONTH FROM base.dataMovimentacao) as data, sum(base.valorTotal) as valor_entrada
    from marketcamp.movimentacaofinanceiraentrada as derived
    inner join marketcamp.movimentacaofinanceira as base
    on derived.movimentacaoID=base.Id
    where base.lojaID=@lojaAtiva and DATE(base.dataMovimentacao) between @dateFrom and @dateUntil
    group by data
),

saidas as (
SELECT EXTRACT(YEAR_MONTH FROM base.dataMovimentacao) as data, sum(base.valorParcela) as valor_saida
    from marketcamp.movimentacaofinanceirasaida as derived
    inner join marketcamp.movimentacaofinanceira as base
    on derived.movimentacaoID=base.Id
    where base.lojaID=@lojaAtiva and DATE(base.dataMovimentacao) between @dateFrom and @dateUntil
    group by data
),
resumo as
(
    (select entradas.data as data, entradas.valor_entrada as valor_entrada, 0 as valor_saida from entradas)
    UNION ALL
    (select saidas.data, 0, saidas.valor_saida from saidas)
)

select data, sum(valor_entrada), sum(valor_saida), sum(valor_entrada)- sum(valor_saida) as liquido
from resumo
group by data
order by data;

data,sum(valor_entrada),sum(valor_saida),liquido
202101,3210.00,0.00,3210.00
202103,200.00,0.00,200.00
202104,0.00,850.00,-850.00
202107,0.00,200.00,-200.00


## Query para listar o total de entrada por meio de pagamento em um período especificado.
As variáveis abaixo representam valores enviados pelo código cliente.

In [4]:
SET @lojaAtiva = 1;
SET @dateFrom = '2021-01-01';
SET @dateUntil = '2021-07-31';

SELECT mp.descricao, sum(base.valorTotal) AS entrada FROM movimentacaofinanceira AS base
INNER JOIN movimentacaofinanceiraentrada AS derived ON base.id=derived.movimentacaoID
INNER JOIN meiopagamento AS mp ON mp.id=derived.meiopagamentoId
WHERE base.lojaid=@lojaAtiva AND DATE(base.dataMovimentacao) BETWEEN @dateFrom AND @dateUntil
GROUP BY derived.meiopagamentoid
ORDER BY entrada DESC;

descricao,entrada
PIX,2750.00
Cartao de Credito,660.00


## Relatório das ultimas entradas da loja
A variável abaixo representa valor enviado pelo código cliente.

In [5]:
SET @lojaAtiva = 1;

SELECT consumidor.nome, mp.descricao, base.datamovimentacao, base.valorTotal
from movimentacaofinanceiraentrada as entrada
inner join consumidor on entrada.consumidorId = consumidor.ID
inner join meiopagamento as mp on entrada.meiopagamentoId=mp.ID
inner join movimentacaofinanceira as base on entrada.movimentacaoID=base.ID
order by base.dataMovimentacao DESC limit 5;

nome,descricao,datamovimentacao,valorTotal
Asdrubal,Cartao de Credito,2021-07-11,1200.00
Eivor Varinsdottir,PIX,2021-07-01,330.00
Bayek de Siuá,Cheque,2021-03-04,1500.00
Desmond Miles,PIX,2021-03-04,200.00
Jacob Frye,Boleto,2021-03-03,1200.00


## Relatório das ultimas saídas da loja
A variável abaixo representa valor enviado pelo código cliente.

In [6]:
SET @lojaAtiva = 1;

SELECT fornecedor.nome, funcionario.nome, base.datamovimentacao, base.valorTotal
from movimentacaofinanceirasaida as saida
inner join fornecedor on saida.fornecedorId = fornecedor.ID
inner join funcionario on saida.aprovadorId=funcionario.ID
inner join movimentacaofinanceira as base on saida.movimentacaoID=base.ID
order by base.dataMovimentacao DESC limit 5;

nome,nome,datamovimentacao,valorTotal
Baragobango LTDA,Paulo Sergio,2021-07-31,100.00
Baragobango LTDA,Trevor Phillips,2021-07-31,100.00
Maria foi com Outras CIA,Sidnei Jhonson,2021-07-30,200.00
Maria foi com Outras CIA,Paulo Sergio,2021-07-30,200.00
Gomes dACosta LTDA,Trevor Phillips,2021-07-11,1200.00


## Valores totais por meio de pagamento

In [12]:
SELECT l.nomeexibicao, mp.descricao, sum(mf.valortotal) as vt
FROM 
  movimentacaofinanceira mf 
JOIN movimentacaofinanceiraentrada mfe ON mf.id = mfe.movimentacaoid
JOIN meiopagamento mp ON mfe.meiopagamentoid = mp.id
JOIN loja l ON mf.lojaid = l.id
GROUP BY l.nomeexibicao;

nomeexibicao,descricao,vt
Casas São Paulo,PIX,3410.00
Gigabyte Comercio Eletronico,Cartao de Credito,1600.00
Kapow,PIX,1400.00
Luiza Magazine,Carteira Digital,900.00
Mercado Preso,Boleto,2700.00
PILOTANDO GOSTOSO,PIX,1530.00
